In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Telecom Customer Churn Prediction\n",
    "Complete the `ChurnPredictor` class below according to the instructions in `instruction.md`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "\n",
    "class ChurnPredictor:\n",
    "    def __init__(self):\n",
    "        \"\"\"Initialize model.\"\"\"\n",
    "        # Initialize your model components here\n",
    "        pass\n",
    "\n",
    "    def fit(self, X, y):\n",
    "        \"\"\"Fit the model.\"\"\"\n",
    "        # Implement preprocessing, encoding, imputation, and training\n",
    "        pass\n",
    "\n",
    "    def predict(self, X):\n",
    "        \"\"\"Predict churn probabilities.\"\"\"\n",
    "        # Implement preprocessing and prediction\n",
    "        # Must return a numpy array or pandas Series\n",
    "        return np.zeros(len(X)) # Placeholder\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}